In [1]:
import torch
import numpy as np

project_dir = os.path.split(os.getcwd())[0]
if project_dir not in sys.path:
    sys.path.append(project_dir)

from models.Paper import CNN
from InformationPlane import TensorKernel, InformationPlane

In [2]:
a = torch.rand((4,1,3,3), dtype=torch.float32)
b = torch.zeros((4,3), dtype=torch.float32)
b[0,1] = 1; b[1,2] = 1; b[2,0] = 1; b[3,1] = 1

print(a)
print(b)

tensor([[[[0.6841, 0.5151, 0.7895],
          [0.8374, 0.8241, 0.0125],
          [0.8338, 0.9763, 0.0990]]],


        [[[0.1473, 0.5664, 0.7274],
          [0.4248, 0.1794, 0.5112],
          [0.9822, 0.1703, 0.6866]]],


        [[[0.9035, 0.6938, 0.1934],
          [0.5990, 0.9495, 0.8394],
          [0.4295, 0.5775, 0.6440]]],


        [[[0.9161, 0.1182, 0.7393],
          [0.6121, 0.2502, 0.2034],
          [0.2182, 0.6957, 0.5666]]]])
tensor([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.]])


In [3]:
a_flatten = torch.flatten(a, 1)

# Tensor kernels definitions

def RBF(x, y, sigma):
    distance = torch.dist(x,y)
    return torch.exp(-(distance**2)/(sigma**2))

def RBF_2(x, sigma):
    distance = torch.cdist(x, x)
    return torch.exp(-(distance**2)/(sigma**2))

# Ambas funciones dan el mismo resultado, lo doy por bueno


In [4]:
rbf_a = RBF_2(a_flatten, 8)/len(a_flatten)
rbf_b = RBF_2(b, 0.9)/len(b)

print(rbf_a)
print(rbf_b)

tensor([[0.2500, 0.2418, 0.2430, 0.2449],
        [0.2418, 0.2500, 0.2420, 0.2431],
        [0.2430, 0.2420, 0.2500, 0.2439],
        [0.2449, 0.2431, 0.2439, 0.2500]])
tensor([[0.2500, 0.0212, 0.0212, 0.2500],
        [0.0212, 0.2500, 0.0212, 0.0212],
        [0.0212, 0.0212, 0.2500, 0.0212],
        [0.2500, 0.0212, 0.0212, 0.2500]])


In [5]:
k_0 = torch.zeros((len(a_flatten), len(a_flatten)))
for i in range(len(a_flatten)):
    for j in range(len(a_flatten)):
        k_0[i,j] = RBF(a_flatten[i], a_flatten[j], 8)
        
print(k_0)

tensor([[1.0000, 0.9670, 0.9721, 0.9798],
        [0.9670, 1.0000, 0.9681, 0.9723],
        [0.9721, 0.9681, 1.0000, 0.9755],
        [0.9798, 0.9723, 0.9755, 1.0000]])


In [6]:
k_1 = torch.zeros((len(b), len(b)))
for i in range(len(b)):
    for j in range(len(b)):
        k_1[i,j] = RBF(b[i], b[j], 0.1)
        
print(k_1)

tensor([[1., 0., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 1.]])


# Kernel Aligment Loss

In [7]:
def kernelAligmentLoss(x, y):
    return ((torch.sum(x*y))/(torch.norm(x) * torch.norm(y))).item()

In [8]:
kernelAligmentLoss(rbf_a, rbf_b)

0.7025044560432434

# Optimize sigma value

In [27]:
mean_distance = torch.tensor([torch.dist(a_flatten[i-1], a_flatten[i]) for i in range(1, len(a_flatten))]).mean()

distance = torch.cdist(a_flatten, a_flatten)
mean_distance_v2 = distance[distance != 0].mean()

print(a_flatten)
print(mean_distance)
print(mean_distance_v2)

possible_sigma_values = np.arange(mean_distance*0.1, mean_distance*10, (mean_distance*10 - mean_distance*0.1)/75)
possible_sigma_values_v2 = np.arange(mean_distance_v2*0.1, mean_distance_v2*10, (mean_distance_v2*10 - mean_distance_v2*0.1)/75)

print(possible_sigma_values)
print(possible_sigma_values_v2)

# print(possible_sigma_values_v2.tolist())

test = list(map(lambda sigma: RBF_2(a_flatten, sigma), possible_sigma_values_v2.tolist()))
# print(test[70])

label_kernel = rbf_b
result = np.array(list(map(lambda x: kernelAligmentLoss(x, label_kernel), test)))
# print(result)

where = np.argwhere(result == result.max()).item(0)
print("---"*3)
print(where)

best_sigma_value = possible_sigma_values_v2[where]
# print(best_sigma_value)

tensor([[0.6841, 0.5151, 0.7895, 0.8374, 0.8241, 0.0125, 0.8338, 0.9763, 0.0990],
        [0.1473, 0.5664, 0.7274, 0.4248, 0.1794, 0.5112, 0.9822, 0.1703, 0.6866],
        [0.9035, 0.6938, 0.1934, 0.5990, 0.9495, 0.8394, 0.4295, 0.5775, 0.6440],
        [0.9161, 0.1182, 0.7393, 0.6121, 0.2502, 0.2034, 0.2182, 0.6957, 0.5666]])
tensor(1.3888)
tensor(1.3329)
[ 0.13887542  0.322191    0.50550658  0.68882215  0.87213773  1.0554533
  1.23876888  1.42208445  1.60540003  1.7887156   1.97203118  2.15534675
  2.33866233  2.5219779   2.70529348  2.88860905  3.07192463  3.2552402
  3.43855578  3.62187135  3.80518693  3.9885025   4.17181808  4.35513365
  4.53844923  4.7217648   4.90508038  5.08839595  5.27171153  5.4550271
  5.63834268  5.82165825  6.00497383  6.1882894   6.37160498  6.55492055
  6.73823613  6.9215517   7.10486728  7.28818285  7.47149843  7.654814
  7.83812958  8.02144516  8.20476073  8.38807631  8.57139188  8.75470746
  8.93802303  9.12133861  9.30465418  9.48796976  9.67128533  

In [11]:
rbf_c = TensorKernel.RBF(a_flatten, 8)
I = InformationPlane(rbf_c, rbf_b)
print(rbf_c)
rbf_c[0, 0] = 0
print(rbf_c)
print(I.input_kernel)

tensor([[1.0000, 0.9670, 0.9721, 0.9798],
        [0.9670, 1.0000, 0.9681, 0.9723],
        [0.9721, 0.9681, 1.0000, 0.9755],
        [0.9798, 0.9723, 0.9755, 1.0000]])
tensor([[0.0000, 0.9670, 0.9721, 0.9798],
        [0.9670, 1.0000, 0.9681, 0.9723],
        [0.9721, 0.9681, 1.0000, 0.9755],
        [0.9798, 0.9723, 0.9755, 1.0000]])
tensor([[0.0000, 0.9670, 0.9721, 0.9798],
        [0.9670, 1.0000, 0.9681, 0.9723],
        [0.9721, 0.9681, 1.0000, 0.9755],
        [0.9798, 0.9723, 0.9755, 1.0000]])


In [12]:
print(I.optimizeSigmaValue(a_flatten))

1.1889302283525467


In [13]:
y = None
x = 0
x = 1 if not(y is None) else x
print(x)

0


In [14]:
print(rbf_a)
print(torch.eig(rbf_a))

tensor([[0.2500, 0.2418, 0.2430, 0.2449],
        [0.2418, 0.2500, 0.2420, 0.2431],
        [0.2430, 0.2420, 0.2500, 0.2439],
        [0.2449, 0.2431, 0.2439, 0.2500]])
torch.return_types.eig(
eigenvalues=tensor([[0.9793, 0.0000],
        [0.0048, 0.0000],
        [0.0071, 0.0000],
        [0.0087, 0.0000]]),
eigenvectors=tensor([]))


In [15]:
import numpy as np
from numpy import linalg as LA
# A = np.array([[1,2,3],[3,2,1],[1,0,-1]])
A = rbf_a.numpy()
print(A)

w, v = LA.eig(A)
print(w)

print(v)


[[0.25       0.24175647 0.24302378 0.2449473 ]
 [0.24175647 0.25       0.2420154  0.24306253]
 [0.24302378 0.2420154  0.25       0.2438743 ]
 [0.2449473  0.24306253 0.2438743  0.25      ]]
[0.97934324 0.00484201 0.00713533 0.00867941]
[[ 0.500201   -0.5513737  -0.49209008 -0.45125753]
 [ 0.49870908 -0.13816388 -0.1157353   0.84782386]
 [ 0.49978063 -0.12583424  0.8331146  -0.20076117]
 [ 0.5013058   0.81305814 -0.22443858 -0.19301881]]


In [16]:
def matrixRenyiEntropy(A : np.array):
    w, _ = LA.eig(A)
    epsilon = 1e-10
    w += epsilon
    entropy = -np.sum(w * np.log2(w))
    
    return entropy

def jointRenyiEntropy(x, y):
    aux = x*y
    return matrixRenyiEntropy(aux/np.trace(aux))
    

In [17]:
o = matrixRenyiEntropy(rbf_a.numpy())
p = matrixRenyiEntropy(rbf_b.numpy())
q = jointRenyiEntropy(rbf_a.numpy(), rbf_b.numpy())
print(o)
print(p)
print(q)

print(o + p - q)

0.17704624
1.490485
1.5317361
0.13579512


In [18]:
print(rbf_a)
print(torch.trace(rbf_a))
print(rbf_b)
aux = rbf_a * rbf_b
print(aux)
print(torch.trace(aux))
print(aux/torch.trace(aux))

tensor([[0.2500, 0.2418, 0.2430, 0.2449],
        [0.2418, 0.2500, 0.2420, 0.2431],
        [0.2430, 0.2420, 0.2500, 0.2439],
        [0.2449, 0.2431, 0.2439, 0.2500]])
tensor(1.)
tensor([[0.2500, 0.0212, 0.0212, 0.2500],
        [0.0212, 0.2500, 0.0212, 0.0212],
        [0.0212, 0.0212, 0.2500, 0.0212],
        [0.2500, 0.0212, 0.0212, 0.2500]])
tensor([[0.0625, 0.0051, 0.0051, 0.0612],
        [0.0051, 0.0625, 0.0051, 0.0051],
        [0.0051, 0.0051, 0.0625, 0.0052],
        [0.0612, 0.0051, 0.0052, 0.0625]])
tensor(0.2500)
tensor([[0.2500, 0.0205, 0.0206, 0.2449],
        [0.0205, 0.2500, 0.0205, 0.0206],
        [0.0206, 0.0205, 0.2500, 0.0206],
        [0.2449, 0.0206, 0.0206, 0.2500]])


In [31]:
a = torch.rand((2,2,2))
b = torch.rand((2,4))
print(len(a.shape))

3


In [35]:
print(a)
shape = a.shape
a = a.flatten(1)
print(a)
a = a.reshape(shape)
print(a)

tensor([[[0.1457, 0.0186],
         [0.6463, 0.8576]],

        [[0.5854, 0.9182],
         [0.5042, 0.4086]]])
tensor([[0.1457, 0.0186, 0.6463, 0.8576],
        [0.5854, 0.9182, 0.5042, 0.4086]])
tensor([[[0.1457, 0.0186],
         [0.6463, 0.8576]],

        [[0.5854, 0.9182],
         [0.5042, 0.4086]]])


In [33]:
print(b)
print(b.flatten(1))

tensor([[0.7791, 0.1968, 0.8002, 0.0073],
        [0.6714, 0.8587, 0.4921, 0.4997]])
tensor([[0.7791, 0.1968, 0.8002, 0.0073],
        [0.6714, 0.8587, 0.4921, 0.4997]])
